In [1]:
from datetime import datetime
import pandas as pd
from trainingData import trainingData

predictionSymbol = '^GSPC' #'^GSPC' #'^IXIC' #symbol for S&P500 (^GSPC) or NASDAQ (^IXIC) or AUS ('^AXJO')
startdate = datetime(2013,10,1)
enddate = datetime(2016,10,11)
teststartdate = datetime(2016,7,1)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)

got nasdaq
got sp500
got volatility
got australia
got hkong


In [2]:
#ds = td.loadAllFromWeb(predictionSymbol, startdate, enddate)

In [3]:
#for column in ds[0].columns:
#    ds[0][column] = ds[0][column].shift(1)
#ds[0]

In [4]:
#import featureGenerator as fg
#wfeat = fg.applyRollMeanDelayedReturns(ds, delta)

In [5]:
testsize = 0.1
X_train, y_train, X_test, y_test = td.returnDataForClassification(teststartdate, testsize)

In [6]:
y_test.size

69

In [7]:
# score accuracy of each algorithm
import algorithms as alg
accuracy, rfc = alg.performRFClass(X_train, y_train, X_test, y_test)
print('RFC =', accuracy)
accuracy, knn = alg.performKNNClass(X_train, y_train, X_test, y_test)
print('KNN =', accuracy)
accuracy, svm = alg.performSVMClass(X_train, y_train, X_test, y_test)
print('SVM =', accuracy)
accuracy, ada = alg.performAdaBoostClass(X_train, y_train, X_test, y_test)
print('ADA =', accuracy)
accuracy, gtb = alg.performGTBClass(X_train, y_train, X_test, y_test)
print('GTB =', accuracy)
#accuracy = alg.performQDAClass(X_train, y_train, X_test, y_test)
#print('QDA =', accuracy)
accuracy, mlp = alg.performMLPClass(X_train, y_train, X_test, y_test, (5,6,2))
print('MLP =', accuracy)

RFC = 0.550724637681
KNN = 0.565217391304
SVM = 0.579710144928
ADA = 0.536231884058
GTB = 0.507246376812
MLP = 0.608695652174


In [9]:
# evaluate for single predictions across all algorithms
startdate = datetime(2016,10,1)
enddate = datetime(2016,10,12)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)
predData = td.returnDataForPrediction()

got nasdaq
got sp500
got volatility
got australia
got hkong


In [31]:
predData = predData[:1]
predData

OutTime2  OutRolMean2  OutTime3  OutRolMean3  OutTime4  \
Date                                                                 
2016-10-07 -0.002773    -0.001386  0.001511     0.000508 -0.003452   

            OutRolMean4  Return_^AXJO  ^AXJOTime2  ^AXJORolMean2  ^AXJOTime3  \
Date                                                                           
2016-10-07    -0.000858      0.001463   -0.001386      -0.000691    0.004126   

                ...       ^HSITime3  ^HSIRolMean3  ^HSITime4  ^HSIRolMean4  \
Date            ...                                                          
2016-10-07      ...       -0.010038     -0.003325  -0.005906      -0.00145   

            Return_Out2  Return_^AXJO2  Return_^IXIC2  Return_^GSPC2  \
Date                                                                   
2016-10-07     0.000482        0.00552       0.004983       0.004297   

            Return_^VIX2  Return_^HSI2  
Date                                    
2016-10-07     -0.046955      0.006902  

[1 rows x 47 columns]

In [30]:

#td.finalTrainingData.columns[1:].size
prediction = rfc.predict(predData)
print('rfc prediction =', prediction)
prediction = knn.predict(predData)
print('knn prediction =', prediction)
prediction = svm.predict(predData)
print('svm prediction =', prediction)
prediction = ada.predict(predData)
print('ada prediction =', prediction)
prediction = gtb.predict(predData)
print('gtb prediction =', prediction)
prediction = mlp.predict(predData)
print('mlp prediction =', prediction)

rfc prediction = [1]
knn prediction = [0]
svm prediction = [1]
ada prediction = [1]
gtb prediction = [1]
mlp prediction = [1]


# Back test

In [8]:
from datetime import datetime
import pandas as pd
from trainingData import trainingData

predictionSymbol = '^GSPC' #'^GSPC' #'^IXIC' #symbol for S&P500 (^GSPC) or NASDAQ (^IXIC) or AUS ('^AXJO')

startdate = datetime(2016,1,10)
enddate = datetime(2016,10,12)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)
predData = td.returnDataForPrediction()

# symbol of the stock required for future plotting
symbol = 'S&P-500'

# name of the file of the output of prediction (S&P 500 in this case)
name = 'sp500-2016.csv'

# calls the best model previously saved in pickle file and runs it on the test set retutning an array of 0,1 (Down, Up) according to predicted returns
#prediction = pystocks.getPredictionFromBestModel(9, 9, 'sp500', cut, start_test, path_datasets, 'sp500_57.pickle')[0]
prediction = mlp.predict(predData)

# dataframe of S&P 500 historical prices (saved locally from Yahho Finance)
bars = pd.read_csv(name, index_col=0, parse_dates=True)    

# subset of the data corresponding to test set
bars = bars[startdate:enddate]

# initialize empty dataframe indexed as the bars. There's going to be perfect match between dates in bars and signals 
signals = pd.DataFrame(index=bars.index)

# initialize signals.signal column to zero
signals['signal'] = 0.0

# copying into signals.signal column results of prediction
signals['signal'] = prediction

# replace the zeros with -1 (new encoding for Down day)
signals.signal[signals.signal == 0] = -1

# compute the difference between consecutive entries in signals.signal. As
# signals.signal was an array of 1 and -1 return signals.positions will 
# be an array of 0s and 2s.
signals['positions'] = signals['signal'].diff()     


got nasdaq
got sp500
got volatility
got australia
got hkong


In [9]:
from portfolio import MarketIntradayPortfolio

# calling portfolio evaluation on signals (predicted returns) and bars 
# (actual returns)
portfolio = MarketIntradayPortfolio(symbol, bars, signals)

# backtesting the portfolio and generating returns on top of that 
returns = portfolio.backtest_portfolio()
returns

KeyError: 'Close_Out'